# GameTheory 21 - Jeux Cooperatifs en Lean : Formalisation de Shapley

**Kernel** : Lean 4 (WSL) - Notebooks Lean natifs

**Navigation** : [← GameTheory-20-Lean-SocialChoice](GameTheory-20-Lean-SocialChoice.ipynb) | [Index](GameTheory-1-Setup.ipynb) | [Fin de la serie]

---

## Introduction

Ce notebook formalise les **jeux cooperatifs** et la **valeur de Shapley** en Lean 4. Ces concepts, introduits dans le notebook 14 (Python), sont ici traites avec la rigueur des preuves formelles.

### Contenu

1. **Jeux cooperatifs (TU games)** : Fonction caracteristique v(S)
2. **Axiomes de Shapley** : Efficacite, symetrie, additivite, joueur nul
3. **Valeur de Shapley** : Definition et unicite
4. **Le Core** : Stabilite des allocations
5. **Jeux de vote** : Indice de Banzhaf

### Lien avec les notebooks precedents

| Notebook | Contenu |
|----------|--------|
| **14 (Python)** | Implementation : calcul exact/Monte Carlo, exemple politique |
| **20 (Lean)** | Social choice : Arrow, Sen, electeur median |
| **21 (Lean)** | Formalisation : axiomes, theoreme d'unicite de Shapley |

### Prerequis

- Notebook 14 (CooperativeGames en Python)
- Notebooks 17-20 (bases Lean, logique, choix social)
- Familiarite avec les types finis (Finset, Fintype)

### Duree estimee : 60 minutes

---

## Plan de ce Notebook

1. [Configuration](#1-configuration)
2. [Jeux Cooperatifs (TU Games)](#2-tu-games)
3. [Axiomes de Shapley](#3-axiomes)
4. [Valeur de Shapley](#4-shapley)
5. [Le Core](#5-core)
6. [Jeux de Vote](#6-voting)
7. [Exercices](#7-exercices)
8. [Solutions](#8-solutions)
9. [Resume](#9-resume)

In [1]:
# Configuration lean_runner

import sys
import os

lean_path = os.path.abspath('../SymbolicAI/Lean')
if lean_path not in sys.path:
    sys.path.insert(0, lean_path)

from lean_runner import LeanRunner

runner = LeanRunner(backend='auto', timeout=60)

def run_lean(code: str, show_code: bool = True):
    """Execute du code Lean et affiche le resultat."""
    result = runner.run(code)
    if show_code:
        print("--- Code Lean ---")
        for i, line in enumerate(code.strip().split('\n'), 1):
            print(f"{i:3d} | {line}")
        print("-" * 40)
    if result.output:
        print("Sortie:", result.output)
    if result.errors:
        print("Erreurs:" if not result.success else "Avertissements:", result.errors)
    print(f"[{'OK' if result.success else 'ECHEC'}]")
    return result

print(f"Backend: {runner.check_installation()['backend']}")
print("\nPret pour executer du code Lean.")

ModuleNotFoundError: No module named 'lean_runner'

<a id="2-tu-games"></a>
## 2. Jeux Cooperatifs (TU Games)

### 2.1 Definition

Un **jeu cooperatif a utilite transferable** (TU game) est defini par :
- Un ensemble fini de joueurs $N = \{1, 2, ..., n\}$
- Une **fonction caracteristique** $v : 2^N \to \mathbb{R}$ avec $v(\emptyset) = 0$

La valeur $v(S)$ represente ce que la coalition $S$ peut obtenir en cooperant.

In [ ]:
run_lean("""
-- Jeu cooperatif a utilite transferable (TU game)

-- Type des joueurs (fini)
variable {N : Type*} [Fintype N] [DecidableEq N]

-- Coalition = sous-ensemble de joueurs
def Coalition (N : Type*) := Finset N

-- Fonction caracteristique : v(S) donne la valeur d'une coalition
structure TUGame (N : Type*) [Fintype N] where
  v : Finset N \u2192 \u211d
  empty_zero : v \u2205 = 0

#check TUGame
#check @TUGame.v
""")

### 2.2 Proprietes structurelles

In [ ]:
run_lean("""
-- Proprietes des jeux cooperatifs

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Superadditivite : la cooperation est benefique
-- v(S \u222a T) \u2265 v(S) + v(T) pour S, T disjoints
def Superadditive (G : TUGame N) : Prop :=
  \u2200 S T : Finset N, Disjoint S T \u2192 
    G.v (S \u222a T) \u2265 G.v S + G.v T

-- Convexite (supermodularite) : contributions marginales croissantes
-- v(S \u222a {i}) - v(S) \u2264 v(T \u222a {i}) - v(T) pour S \u2286 T, i \u2209 T
def Convex (G : TUGame N) : Prop :=
  \u2200 S T : Finset N, \u2200 i : N, 
    S \u2286 T \u2192 i \u2209 T \u2192
    G.v (S \u222a {i}) - G.v S \u2264 G.v (T \u222a {i}) - G.v T

#check @Superadditive
#check @Convex
""")

### 2.3 Exemples classiques

In [ ]:
run_lean("""
-- Exemples de jeux cooperatifs

-- Jeu d'unanimite : v(S) = 1 si T \u2286 S, 0 sinon
-- Les joueurs de T doivent tous etre presents pour creer de la valeur
def unanimityGame {N : Type*} [Fintype N] [DecidableEq N] 
    (T : Finset N) : TUGame N := {
  v := fun S => if T \u2286 S then 1 else 0
  empty_zero := by
    simp only
    by_cases hT : T = \u2205
    \u00b7 simp [hT]
    \u00b7 simp only [Finset.not_subset_empty_iff.mpr hT, ite_false]
}

-- Jeu de majorite simple : v(S) = 1 si |S| > n/2, 0 sinon
def majorityGame {N : Type*} [Fintype N] [DecidableEq N] : TUGame N := {
  v := fun S => if 2 * S.card > Fintype.card N then 1 else 0
  empty_zero := by simp
}

#check @unanimityGame
#check @majorityGame
""")

<a id="3-axiomes"></a>
## 3. Axiomes de Shapley

Shapley (1953) a montre que quatre axiomes caracterisent de maniere unique la valeur de Shapley :

1. **Efficacite** : La somme des paiements egale la valeur de la grande coalition
2. **Symetrie** : Joueurs interchangeables recoivent le meme paiement
3. **Joueur nul** : Un joueur sans contribution marginale recoit 0
4. **Additivite** : La valeur de la somme de deux jeux = somme des valeurs

In [ ]:
run_lean("""
-- Axiomes de Shapley pour une solution \u03c6

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Une solution : associe a chaque jeu un vecteur de paiements
def Solution (N : Type*) [Fintype N] := TUGame N \u2192 (N \u2192 \u211d)

-- Axiome 1 : Efficacite
-- La somme des paiements = valeur de la grande coalition
def Efficiency (\u03c6 : Solution N) : Prop :=
  \u2200 G : TUGame N, \u2211 i, \u03c6 G i = G.v Finset.univ

-- Axiome 2 : Symetrie
-- Deux joueurs symetriques recoivent le meme paiement
def Symmetric (G : TUGame N) (i j : N) : Prop :=
  \u2200 S : Finset N, i \u2209 S \u2192 j \u2209 S \u2192
    G.v (S \u222a {i}) = G.v (S \u222a {j})

def SymmetryAxiom (\u03c6 : Solution N) : Prop :=
  \u2200 G : TUGame N, \u2200 i j : N, 
    Symmetric G i j \u2192 \u03c6 G i = \u03c6 G j

-- Axiome 3 : Joueur nul
-- Un joueur avec contribution marginale nulle recoit 0
def NullPlayer (G : TUGame N) (i : N) : Prop :=
  \u2200 S : Finset N, i \u2209 S \u2192 
    G.v (S \u222a {i}) = G.v S

def NullPlayerAxiom (\u03c6 : Solution N) : Prop :=
  \u2200 G : TUGame N, \u2200 i : N,
    NullPlayer G i \u2192 \u03c6 G i = 0

-- Axiome 4 : Additivite
-- \u03c6(G + H) = \u03c6(G) + \u03c6(H)
def AddGames (G H : TUGame N) : TUGame N := {
  v := fun S => G.v S + H.v S
  empty_zero := by simp [G.empty_zero, H.empty_zero]
}

def AdditivityAxiom (\u03c6 : Solution N) : Prop :=
  \u2200 G H : TUGame N, \u2200 i : N,
    \u03c6 (AddGames G H) i = \u03c6 G i + \u03c6 H i

#check @Efficiency
#check @SymmetryAxiom
#check @NullPlayerAxiom
#check @AdditivityAxiom
""")

<a id="4-shapley"></a>
## 4. Valeur de Shapley

### 4.1 Definition

La valeur de Shapley du joueur $i$ est sa contribution marginale moyenne sur toutes les permutations :

$$\phi_i(v) = \sum_{S \subseteq N \setminus \{i\}} \frac{|S|!(n-|S|-1)!}{n!} [v(S \cup \{i\}) - v(S)]$$

In [ ]:
run_lean("""
-- Definition de la valeur de Shapley

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Contribution marginale de i a la coalition S
def marginalContribution (G : TUGame N) (i : N) (S : Finset N) : \u211d :=
  G.v (S \u222a {i}) - G.v S

-- Coefficient de Shapley pour une coalition de taille s sur n joueurs
-- = s! * (n-s-1)! / n!
noncomputable def shapleyCoef (n s : \u2115) : \u211d :=
  (Nat.factorial s * Nat.factorial (n - s - 1)) / Nat.factorial n

-- Valeur de Shapley (simplifiee, necessite Mathlib pour calcul complet)
noncomputable def shapleyValue (G : TUGame N) (i : N) : \u211d :=
  \u2211 S in (Finset.univ.filter (fun S => i \u2209 S)),
    shapleyCoef (Fintype.card N) S.card * marginalContribution G i S

#check @marginalContribution
#check @shapleyValue
""")

### 4.2 Theoreme d'unicite

In [ ]:
run_lean("""
-- Theoreme de Shapley : la valeur de Shapley est l'unique solution
-- satisfaisant les quatre axiomes

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Enonce du theoreme d'unicite (version simplifiee)
theorem shapley_uniqueness_sketch (\u03c6 : Solution N)
    (h_eff : Efficiency \u03c6)
    (h_sym : SymmetryAxiom \u03c6)
    (h_null : NullPlayerAxiom \u03c6)
    (h_add : AdditivityAxiom \u03c6) :
    -- Alors \u03c6 est la valeur de Shapley
    \u2200 G : TUGame N, \u2200 i : N, \u03c6 G i = shapleyValue G i := by
  -- La preuve utilise la decomposition en jeux d'unanimite
  -- Tout jeu G peut s'ecrire comme somme de jeux d'unanimite
  -- G = \u2211_T c_T * u_T
  -- Par additivite : \u03c6(G) = \u2211_T c_T * \u03c6(u_T)
  -- Par symetrie et efficacite : \u03c6_i(u_T) = 1/|T| si i \u2208 T, 0 sinon
  sorry

#check @shapley_uniqueness_sketch
""")

### 4.3 Proprietes de la valeur de Shapley

In [ ]:
run_lean("""
-- Proprietes de la valeur de Shapley

variable {N : Type*} [Fintype N] [DecidableEq N]

-- La valeur de Shapley satisfait l'efficacite (a prouver)
theorem shapley_efficient : \u2200 G : TUGame N,
    \u2211 i, shapleyValue G i = G.v Finset.univ := by
  sorry

-- Pour les jeux convexes, Shapley est dans le Core
-- (voir section 5)

-- Pour les jeux d'unanimite, la valeur a une forme simple
theorem shapley_unanimity (T : Finset N) (i : N) :
    shapleyValue (unanimityGame T) i = 
    if i \u2208 T then (1 : \u211d) / T.card else 0 := by
  sorry

#check @shapley_efficient
#check @shapley_unanimity
""")

<a id="5-core"></a>
## 5. Le Core

Le **Core** d'un jeu cooperatif est l'ensemble des allocations stables : aucune coalition ne peut faire mieux en se separant.

In [ ]:
run_lean("""
-- Le Core d'un jeu cooperatif

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Une allocation est un vecteur de paiements
def Allocation (N : Type*) := N \u2192 \u211d

-- Le Core : allocations efficaces et stables
def Core (G : TUGame N) : Set (Allocation N) :=
  { x | 
    -- Efficacite : la somme = valeur de N
    (\u2211 i, x i = G.v Finset.univ) \u2227
    -- Rationalite de groupe : aucune coalition ne peut bloquer
    (\u2200 S : Finset N, \u2211 i in S, x i \u2265 G.v S)
  }

-- Le Core peut etre vide !
def CoreEmpty (G : TUGame N) : Prop := Core G = \u2205

#check @Core
#check @CoreEmpty
""")

In [ ]:
run_lean("""
-- Theoreme de Bondareva-Shapley
-- Le Core est non-vide ssi le jeu est equilibre

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Collection equilibree (balanced collection)
-- Poids \u03bb_S tels que pour tout i : \u2211_{S \u220b i} \u03bb_S = 1
def Balanced (weights : Finset N \u2192 \u211d) : Prop :=
  (\u2200 S, weights S \u2265 0) \u2227
  (\u2200 i : N, \u2211 S in (Finset.univ.filter (fun S => i \u2208 S)), weights S = 1)

-- Jeu equilibre : pour toute collection equilibree,
-- \u2211_S \u03bb_S * v(S) \u2264 v(N)
def BalancedGame (G : TUGame N) : Prop :=
  \u2200 weights : Finset N \u2192 \u211d,
    Balanced weights \u2192
    \u2211 S in Finset.univ, weights S * G.v S \u2264 G.v Finset.univ

-- Theoreme de Bondareva-Shapley (enonce)
theorem bondareva_shapley (G : TUGame N) :
    (Core G).Nonempty \u2194 BalancedGame G := by
  -- La preuve utilise le theoreme de dualite en programmation lineaire
  sorry

#check @bondareva_shapley
""")

In [ ]:
run_lean("""
-- Pour les jeux convexes, Shapley est dans le Core

variable {N : Type*} [Fintype N] [DecidableEq N]

theorem convex_shapley_in_core (G : TUGame N) (h : Convex G) :
    (fun i => shapleyValue G i) \u2208 Core G := by
  -- Les jeux convexes ont un Core non-vide
  -- et Shapley y appartient (extremal point)
  sorry

#check @convex_shapley_in_core
""")

<a id="6-voting"></a>
## 6. Jeux de Vote

Les jeux de vote sont des cas particuliers ou $v(S) \in \{0, 1\}$ (une coalition gagne ou perd).

In [ ]:
run_lean("""
-- Jeux de vote ponderes

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Jeu de vote pondere [q; w1, w2, ..., wn]
-- Une coalition gagne si la somme des poids >= quota
def WeightedVotingGame (weights : N \u2192 \u211d) (quota : \u211d) : TUGame N := {
  v := fun S => if \u2211 i in S, weights i \u2265 quota then 1 else 0
  empty_zero := by simp
}

-- Un joueur est critique dans une coalition si son retrait la fait perdre
def Critical (G : TUGame N) (i : N) (S : Finset N) : Prop :=
  i \u2208 S \u2227 G.v S = 1 \u2227 G.v (S.erase i) = 0

-- Indice de Banzhaf (non normalise)
-- Compte le nombre de coalitions ou i est critique
def BanzhafRaw (G : TUGame N) (i : N) : \u2115 :=
  (Finset.univ.filter (fun S => Critical G i S)).card

#check @WeightedVotingGame
#check @Critical
#check @BanzhafRaw
""")

In [ ]:
run_lean("""
-- Proprietes des joueurs dans les jeux de vote

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Joueur avec droit de veto : present dans toute coalition gagnante
def VetoPlayer (G : TUGame N) (i : N) : Prop :=
  \u2200 S : Finset N, G.v S = 1 \u2192 i \u2208 S

-- Dictateur : peut gagner seul et personne d'autre ne peut gagner sans lui
def Dictator (G : TUGame N) (i : N) : Prop :=
  G.v {i} = 1 \u2227 VetoPlayer G i

-- Joueur nul : jamais critique
def DummyPlayer (G : TUGame N) (i : N) : Prop :=
  \u2200 S : Finset N, i \u2209 S \u2192 G.v (S \u222a {i}) = G.v S

-- Un joueur nul a Shapley = 0 et Banzhaf = 0
theorem dummy_shapley_zero (G : TUGame N) (i : N) :
    DummyPlayer G i \u2192 shapleyValue G i = 0 := by
  intro h
  -- Un joueur nul satisfait NullPlayer, donc Shapley = 0
  sorry

#check @VetoPlayer
#check @Dictator
#check @DummyPlayer
#check @dummy_shapley_zero
""")

<a id="7-exercices"></a>
## 7. Exercices

### Exercice 1 : Jeu de gants

Trois joueurs : L1, L2 ont chacun un gant gauche, R1 a un gant droit.
Une paire de gants vaut 1.

1. Definir la fonction caracteristique
2. Calculer la valeur de Shapley de chaque joueur

### Exercice 2 : Verification d'axiome

Prouver que la valeur de Shapley satisfait l'axiome du joueur nul.

### Exercice 3 : Core vide

Montrer que le jeu de majorite simple a 3 joueurs a un Core vide.

<a id="8-solutions"></a>
## 8. Solutions

### Solution Exercice 1

In [ ]:
run_lean("""
-- Solution Exercice 1 : Jeu de gants

-- Trois joueurs : L1, L2 (gants gauches), R1 (gant droit)
inductive GlovePlayer where
  | L1 | L2 | R1
  deriving DecidableEq, Fintype

open GlovePlayer

def hasLeftGlove (p : GlovePlayer) : Bool :=
  match p with
  | L1 => true
  | L2 => true
  | R1 => false

def hasRightGlove (p : GlovePlayer) : Bool :=
  match p with
  | L1 => false
  | L2 => false
  | R1 => true

-- Fonction caracteristique : nombre de paires completes
def gloveGame : TUGame GlovePlayer := {
  v := fun S => 
    let nLeft := (S.filter (fun p => hasLeftGlove p)).card
    let nRight := (S.filter (fun p => hasRightGlove p)).card
    min nLeft nRight
  empty_zero := by simp
}

-- Valeurs de Shapley (calcul manuel) :
-- R1 a une contribution marginale plus elevee (ressource rare)
-- Shapley(L1) = Shapley(L2) = 1/6
-- Shapley(R1) = 4/6 = 2/3

#check gloveGame
""")

### Solution Exercice 2

In [ ]:
run_lean("""
-- Solution Exercice 2 : Shapley satisfait l'axiome du joueur nul

variable {N : Type*} [Fintype N] [DecidableEq N]

theorem shapley_null_player (G : TUGame N) (i : N) :
    NullPlayer G i \u2192 shapleyValue G i = 0 := by
  intro h_null
  -- Si i est un joueur nul, sa contribution marginale est toujours 0
  -- marginalContribution G i S = v(S \u222a {i}) - v(S) = 0
  -- Donc shapleyValue G i = \u2211_S coef * 0 = 0
  unfold shapleyValue
  apply Finset.sum_eq_zero
  intro S hS
  simp at hS
  have h : marginalContribution G i S = 0 := by
    unfold marginalContribution
    rw [h_null S hS]
    ring
  simp [h]

#check @shapley_null_player
""")

### Solution Exercice 3

In [ ]:
# Solution Exercice 3 : Core vide du jeu de majorite a 3 joueurs

print("JEU DE MAJORITE SIMPLE A 3 JOUEURS")
print("=" * 50)
print("""
Joueurs : {1, 2, 3}
Quota : majorite = 2

Fonction caracteristique :
  v({}) = 0
  v({1}) = v({2}) = v({3}) = 0
  v({1,2}) = v({1,3}) = v({2,3}) = 1
  v({1,2,3}) = 1

Pour qu'une allocation (x1, x2, x3) soit dans le Core :
  1. Efficacite : x1 + x2 + x3 = 1
  2. Stabilite : 
     x1 + x2 >= 1  (coalition {1,2} ne bloque pas)
     x1 + x3 >= 1  (coalition {1,3} ne bloque pas)
     x2 + x3 >= 1  (coalition {2,3} ne bloque pas)

En additionnant les trois contraintes de stabilite :
  2(x1 + x2 + x3) >= 3
  2 * 1 >= 3  (par efficacite)
  2 >= 3  CONTRADICTION!

=> Le Core est VIDE.

Intuition : chaque coalition de 2 peut "bloquer" et demander au moins 1,
mais il n'y a que 1 a partager entre les 3 joueurs.
""")

<a id="9-resume"></a>
## 9. Resume

### Concepts formalises

| Concept | Definition Lean |
|---------|----------------|
| TU Game | `TUGame N` avec `v : Finset N -> R`, `v(empty) = 0` |
| Superadditivite | `v(S u T) >= v(S) + v(T)` pour S, T disjoints |
| Convexite | Contributions marginales croissantes |
| Shapley | Contribution marginale moyenne sur permutations |
| Core | Allocations efficaces et stables |
| Banzhaf | Nombre de coalitions critiques |

### Axiomes de Shapley

| Axiome | Signification |
|--------|---------------|
| Efficacite | `sum phi = v(N)` |
| Symetrie | Joueurs interchangeables = meme paiement |
| Joueur nul | Contribution nulle = paiement nul |
| Additivite | `phi(G+H) = phi(G) + phi(H)` |

### Theoremes cles

1. **Unicite de Shapley** : Les 4 axiomes caracterisent Shapley uniquement
2. **Bondareva-Shapley** : Core non-vide ssi jeu equilibre
3. **Jeux convexes** : Shapley dans le Core

### Lien avec le notebook Python (14)

| Notebook 14 (Python) | Notebook 21 (Lean) |
|---------------------|-------------------|
| Implementation Monte Carlo | Theoreme d'unicite |
| Exemple politique francais | Axiomatisation formelle |
| Calcul numerique | Preuves mathematiques |

---

## Conclusion de la Serie GameTheory

Cette serie de **21 notebooks** a couvert la theorie des jeux de maniere complete :

| Partie | Focus |
|--------|-------|
| 1-6 | Fondations : Nash, minimax, evolution |
| 7-11 | Dynamique : forme extensive, induction, bayesien |
| 12-16 | Algorithmes : CFR, cooperatif, mecanismes, MARL |
| 17-21 | Formalisation : Lean 4, preuves, Shapley |

---

**Navigation** : [<- GameTheory-20-Lean-SocialChoice](GameTheory-20-Lean-SocialChoice.ipynb) | [Index](GameTheory-1-Setup.ipynb) | [Fin de la serie]